In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipaddress
from datetime import datetime
import time
import keras
import re
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer as DV

C:\Users\ASUS\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv("./data_full.csv", sep = ";", error_bad_lines=False)

b'Skipping line 10780784: expected 10 fields, saw 11\nSkipping line 10793876: expected 10 fields, saw 11\n'
b'Skipping line 10821923: expected 10 fields, saw 11\n'
C:\Users\ASUS\Anaconda3\envs\tensorflow-gpu\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
catalog = pd.read_csv("./catalog.csv", sep = ";", error_bad_lines=False)

In [4]:
data.head()

,timestamp,ip,track_id,cookie,live,user_id,referer,uagent,rightholder,author_id
0,1517778000,230.59.74.120,10912114,42536dc7a8578b0cfac05f704977429a,0,NaN,https%3A%2F%2Frutube.ru%2Fvideo%2F0f5c9c5839b1...,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2,1480930
1,1517778000,218.222.225.36,10885813,8f92f3f2b1e7a2498761b8cb3b1d03c3,0,NaN,https%3A%2F%2Frutube.ru%2Fvideo%2Fd8068436dcf5...,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,2,301323
2,1517778000,55.63.241.224,6948236,336d5ebc5436534e61d16e63ddfca327,-,2.231332e+09,http%3A%2F%2Fandroid.rutube.ru%2F,okhttp/2.6.0,-,-
3,1517778000,230.34.98.99,8481402,cf8dd809edf104873a1c57921b34f25e,0,NaN,https%3A%2F%2Frutube.ru%2Fvideo%2F1058cf249e46...,Mozilla/5.0 (Linux; Android 7.1.2; Redmi 4X Bu...,NaN,721346
4,1517778000,232.3.41.226,10908060,4ca525a1d60de6d74e3c32529fe47ab5,0,NaN,https%3A%2F%2Fyastatic.net%2Fvideo-player%2F0x...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,135,599848


In [5]:
catalog.head()

,idec,track_id,video_id,series_id,title,season,episode,episode_global,date_efir,duration
0,PR592505,10258734,cd59084eb50e6ebe9fe20ad04e0c17db,1.045685e+10,"\tБыть или не быть, 1 сезон, 1 серия",1.0,1.0,1.0,02.04.2017 22:00:34,NaN
1,PR609630,10843255,a5d2e6bae4752d9ec0fae9b53f00700d,1.059247e+10,NaN,11.0,1.0,1.0,NaN,NaN
2,PR597128,10573153,bcd45606c1f7cbb5cb9d1dbd7d346cc3,1.047956e+10,NaN,7.0,1.0,1.0,06.09.2017 22:46:05,NaN
3,PR257331,6617650,609217ac7b8c4e55d44c4a29bf52554b,NaN,"""Интерны. История болезни""",NaN,NaN,NaN,12.06.2012 22:00:44,NaN
4,PR529698,7991493,bde3620ba038b3e8dd41d03bf516a7eb,1.024102e+10,#ЖАННАПОЖЕНИ,1.0,1.0,101.0,05.09.2015 13:30:00,NaN


In [6]:
def add_device(df):
    regexpDesktop = re.compile('(Windows|Linux|Mozilla)')
    regexpMobile = re.compile('(iPhone|RutubeAndroid|okhttp|CFNetwork|UCBrowser)')
    regexpTablet = re.compile('(iPad)')
    
    df['device'] = pd.Series('0', index=df.index)
    
    for i, row in df.iterrows():
        try:
            if regexpDesktop.search(df['uagent'].iloc[i]):
                df['device'][i] = 'desktop'
            elif regexpMobile.search(df['uagent'].iloc[i]):
                df['device'][i] = 'mobile'
            elif regexpTablet.search(df['uagent'].iloc[i]):
                df['device'][i] = 'mobile'
        except:
            pass
            
    return df

In [7]:
def mergeData(data1, data2, field):
    data1[field] = pd.to_numeric(data1[field], errors = 'coerce')
    data2[field] = pd.to_numeric(data2[field], errors = 'coerce')
    return pd.merge(data1, data2, left_on=field, right_on=field)

In [8]:
def convIpToNumber(data):
    return data.apply(lambda x: int(ipaddress.IPv4Address(x)))

In [9]:
def applyDate(x):
    try:
        data = time.strptime(x, "%d.%m.%Y %H:%M:%S")
        return int(time.mktime(data))
    except:
        return 1e-6

In [10]:
def confDateStringToTimestamp(data):
    return data.apply(lambda x: applyDate(x))

In [11]:
def filter_users_with_n_views(data, user_column = "ip", seq_len = 4):
    by_user = data.groupby(user_column)
    by_user = by_user.filter(lambda x: len(x) >= seq_len)
    return by_user

def extract_sequences_by_user(data, user_column = "ip"):
    return data.groupby(user_column)

In [12]:
def filter_videos_with_n_views(data, n = 1000):
    data["track_id"] = pd.to_numeric(data["track_id"], errors = 'coerce', downcast = 'integer')
    data = data[data["track_id"].notnull()]
    data_top_n = data.groupby("track_id").filter(lambda x: len(x) >= n)
    return data_top_n

In [13]:
# data['ip'] = convIpToNumber(data['ip'])

In [14]:
# data_top_n = filter_videos_with_n_views(data)
# print(len(data_top_n))

In [15]:
# prepared = filter_users_with_n_views(data_top_n)
# print(len(prepared))

In [16]:
numeric_cols = [ 'series_id', 'device', 'idec', 'author_id']

def label_encoder(data):
    le = preprocessing.LabelEncoder()
    le.fit(data)
    return le

def create_cat_vectorizer(data):
    
    cat = data.drop( numeric_cols, axis = 1 )
    cat.fillna( 'NA', inplace = True )
    x_cat = cat.to_dict( orient = 'records' )
    
    vectorizer = DV( sparse = False )
    vectorizer.fit(x_cat)
    
    return vectorizer

def cat_vectorize(data, vectorizer):
    
    cat = data.drop( numeric_cols, axis = 1 )
    cat.fillna( 'NA', inplace = True )
    x_cat = cat.to_dict( orient = 'records' )
    
    return vectorizer.transform(data)
    

In [20]:
def replace_null_value(x):
    if (x == 'null'):
        return 0
    else:
        return x

def data_scale(data):
    scaler = StandardScaler()
    data.fillna(0, inplace = True )
    data = data.apply(lambda x: replace_null_value(x))
    print(data)
    data = scaler.fit_transform(data.reshape(-1, 1))
    return data

In [21]:
def data_to_num_and_cat(data):
    data['date_efir'] = data_scale(data['date_efir'])
    data['duration'] = data_scale(data['duration'])
    return data

In [ ]:
def prepare_data(data, catalog):
    
    data['ip'] = convIpToNumber(data['ip'])
    
    data = filter_videos_with_n_views(data)
    print(len(data_top_n))
    
    data = filter_users_with_n_views(data)
    print(len(data))
    
    data = mergeData(data, catalog, "track_id")
    
    data['date_efir'] = confDateStringToTimestamp(data['date_efir'])
    
    data = data_to_num_and_cat(data)
    
    return data

prepared = prepare_data(data, catalog)

In [82]:
cat_vectorizer = create_cat_vectorizer(prepared)
print(cat_vectorize(data.head(5), cat_vectorizer))

KeyError: "['series_id' 'device' 'idec'] not in index"

In [ ]:
track_ids = prepared["track_id"]

vocab_size = track_ids.nunique()

le = label_encoder(track_ids.values)

print(le.transform(track_ids.values[0:5]))
print(vocab_size)

In [21]:
def create_x_video_vectors(x, vocab_size):
    x = x["track_id"].values
    x = le.transform(x)
    x = x.astype(np.int32)
#     x = np.expand_dims(x, axis=0).astype(np.int32)
#     x = keras.utils.to_categorical(x, vocab_size)
    return x

    #     return np.repeat(np.expand_dims(x["track_id"].values, axis=0), len(x), axis = 0).reshape(len(x), len(x))

def create_x_num_cut_vectors(x):
    vec_x_cat = cat_vectorizer.transform(x, cat_vectorizer)
    print(vec_x_cat)
    x_num = data[x].as_matrix()
    print(x_num)
    return np.hstack(x_num, vec_x_cat)

def create_x(x, vocab_size):
    return [create_x_video_vectors(x, vocab_size).astype(np.int32), create_x_num_cut_vectors(x)]
    
print(create_x(prepared.head(3), vocab_size))

[858 690 804]


In [22]:
# one_hot = one_hot_encoder(prepared, "track_id", vocab_size)

def create_y(y, vocab_size):
    y = y["track_id"].values
    y = le.transform(y)
    if (len(y) == 0):
        return np.zeros(vocab_size)
#     y = one_hot.transform(y["track_id"][y["track_id"].notnull()].values, axis = 1).toarray()
#     y = np.expand_dims(y, axis=0).astype(np.int32)
    return np.max(keras.utils.to_categorical(y, vocab_size), axis = 0)

print(create_y(prepared.head(3), vocab_size))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [23]:
def steps(data, batch_size = 20):
    return data["ip"].nunique() / batch_size

In [25]:
def train_generator(seq_by_user, x_len = 4, batch_size = 20):
    
    step = 0
    
    seq_len = len(seq_by_user)
    
    x_list = []
    y_list = []
    
    iterator = iter(seq_by_user)
    
    while(True):
        ip, group = next(iterator)
        
        group = group.sort_values("timestamp")
        x = create_x(group[0:x_len], vocab_size)
        y = create_y(group[x_len:min(x_len + 4, len(group))], vocab_size)
        
        x_list.append(x)
        y_list.append(y)
        
        step = step + 1
        
        if (step % batch_size == 0 or step >= seq_len):
            batch = (np.array(x_list), np.array(y_list))
            x_list = []
            y_list = []
            yield (batch)
            
        if (step >= seq_len):
            iterator = iter(seq_by_user)
            step = 0
        
generation = next(train_generator(extract_sequences_by_user(prepared)))
print(generation[0].shape)
print(generation[1].shape)
print(generation[0])
print(generation[1])

(20, 4)
(20, 955)
[[144 146 149 150]
 [752 753 766 781]
 [348 348 726 825]
 [869 869 513 513]
 [888 897 832 810]
 [886 913 937 933]
 [781 844 781 781]
 [783 741 751 348]
 [879 880 913 933]
 [743 766 781 944]
 [159 160 160 161]
 [369 370 372 243]
 [764 944 887 944]
 [860 860 751 764]
 [913 933 937 952]
 [708 703 702 698]
 [860 880 860 906]
 [691 693 697 697]
 [348 348 348 348]
 [860 879 906 913]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# test_seq = extract_sequences_by_user(prepared[:100])
# test_gen = train_generator(test_seq, 3)
# for i in steps(test_seq, 3) * 2:
#     print(next(test_gen))

In [26]:
def add_train_flag(data):
    split_val = prepared["ip"].quantile(.8)
    print(split_val)
    prepared["train"] = prepared["ip"] < split_val
    
def train_test_split(data):
    add_train_flag(data)
    return (prepared[prepared["train"] == True], prepared[prepared["train"] == False])

In [27]:
train, test = train_test_split(prepared)
print(len(train))
print(len(test))

3822537001.0


C:\Users\ASUS\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


3923196
980801


In [28]:
print(steps(train))
print(steps(test))

21179.25
6099.45


In [29]:
# vocab_size = 248868
# print(vocab_size)

emb_input = keras.layers.Input(shape=(4,))

num_cut_input = keras.layers.Input(shape=(500,))

embedding = keras.layers.Embedding(vocab_size, 100, input_length=4)(emb_input)

concat = keras.layers.Concatenate()(num_cut_input, embedding)

# avg = keras.layers.Flatten()(keras.layers.AveragePooling1D(pool_size=1)(embedding))

#input_num_cat = keras.layers.Input(shape = num_cat_shape)
#concat = keras.layers.Concat(embedding, input_num_cat)
gru_1 = keras.layers.CuDNNGRU(128, return_sequences=True)(concat)
gru_2 = keras.layers.CuDNNGRU(128)(gru_1)

dense_1 = keras.layers.Dense(512, activation = "relu")(gru_2)
# dense_2 = keras.layers.Dense(512, activation = "relu")(dense_1)
drop = keras.layers.Dropout(0.5)(dense_1)
dense_4 = keras.layers.Dense(vocab_size, activation = "tanh")(drop)

model = keras.models.Model(inputs=emb_input, outputs=dense_4)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['top_k_categorical_accuracy'])

model.summary()

model.fit_generator(train_generator(extract_sequences_by_user(train), batch_size=50), steps_per_epoch=steps(train, 50), validation_data = train_generator(extract_sequences_by_user(test)), validation_steps=steps(test), epochs=3, verbose=1)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 4, 100)            95500     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 4, 128)            88320     
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (None, 128)               99072     
_________________________________________________________________
dense_1 (Dense)              (None, 512)               66048     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 955)               489915    
Total para

KeyboardInterrupt: 

In [207]:
val_batch = next(train_generator(extract_sequences_by_user(test)))
y_true = val_batch[1]
y_predicted = model.predict(val_batch[0])

print(y_true.shape)
print(y_predicted.shape)

(20, 955)
(20, 955)


In [217]:
y_predicted = np.rint(y_predicted)

true_y_values = np.transpose(np.nonzero(y_true))[:, 1:]
pred_y_values = np.transpose(np.nonzero(y_predicted))[:, :1]

true_y_track_values = le.inverse_transform(true_y_values)
pred_y_track_values = le.inverse_transform(pred_y_values)

# y_true = np.array(np.nonzero(y_true)).flatten()
# y_predicted = np.array(np.nonzero(y_predicted)).flatten()

# print(y_true.shape)
# print(y_predicted.shape)

# def inverse(x):
#     if x > 0 : 
#         return le.inverse_transform(x) 
#     else :
#         return 0
        
# inverse = np.vectorize(inverse)
# y_true = inverse(y_true)
# y_predicted = inverse(y_predicted) if len (y_predicted) > 0 else y_predicted

# print(y_true)
# print(y_predicted)

[[  0 139]
 [  0 887]
 [  0 944]
 [  0 950]
 [  1 332]
 [  1 628]
 [  1 647]
 [  1 887]
 [  2 173]
 [  2 177]
 [  2 196]
 [  2 839]
 [  3 539]
 [  3 887]
 [  4 170]
 [  4 391]
 [  4 680]
 [  4 753]
 [  5 154]
 [  5 702]
 [  5 929]
 [  5 933]
 [  6 139]
 [  6 721]
 [  6 891]
 [  6 944]
 [  7 901]
 [  7 906]
 [  7 933]
 [  7 944]
 [  8 534]
 [  8 676]
 [  8 887]
 [  9 742]
 [  9 743]
 [  9 842]
 [  9 887]
 [ 10  48]
 [ 10 515]
 [ 10 671]
 [ 10 944]
 [ 11 742]
 [ 11 941]
 [ 11 944]
 [ 12 894]
 [ 12 933]
 [ 12 944]
 [ 12 950]
 [ 13 887]
 [ 13 906]
 [ 14 327]
 [ 14 760]
 [ 14 887]
 [ 15 887]
 [ 15 908]
 [ 16 702]
 [ 16 887]
 [ 17 552]
 [ 17 887]
 [ 17 894]
 [ 17 944]
 [ 18 173]
 [ 18 944]
 [ 19 596]
 [ 19 764]
 [ 19 887]
 [ 19 894]]
[[  0 130]
 [  0 150]
 [  0 348]
 ...
 [ 16 949]
 [ 16 950]
 [ 16 952]]


In [ ]:
def get_next_video(track_id, video_catalog):
    video_catalog['season'] = pd.to_numeric(video_catalog['season'], errors='coerce', downcast = 'integer')
    video_catalog['episode'] = pd.to_numeric(video_catalog['episode'], errors='coerce', downcast = 'integer')

    curent_video = video_catalog.loc[video_catalog['track_id'] == track_id]

    if curent_video.size == 0:
        return False

    curent_video_index = curent_video.index[0]
    
    curent_title = curent_video['title'][[curent_video_index][0]]
    curent_season = curent_video['season'][[curent_video_index][0]]
    curent_episod = curent_video['episode'][[curent_video_index][0]]
    
    this_show_season = video_catalog.loc[(video_catalog['season'] == curent_season) & (video_catalog.title == curent_title)]
    next_episods = this_show_season.loc[this_show_season['episode'] > curent_episod]

    if next_episods.episode.size != 0:
        next_min_episode = next_episods.episode.min()
        next_min_episode_index = next_episods.loc[next_episods.episode == next_min_episode].index[0]
        next = video_catalog.ix[next_min_episode_index]
    else:
        next_seasons = video_catalog.loc[(video_catalog['season'] > curent_season) & (video_catalog.title == curent_title)]
        if next_seasons.season.size != 0:
            next_min_season_min_episode = next_seasons.episode.min()
            next = next_seasons.loc[next_seasons['episode'] == next_min_season_min_episode]
        else:
            next = False

    return next

In [ ]:
def get_top_popular_video(video_history):
    return video_history.groupby('track_id').apply(lambda x: x.count()).sort_values(by=['ip'], ascending=False)

In [ ]:
def find_video_id_by_track_id(id): 
    video_id = catalog.loc[catalog['track_id'] == id]['video_id']
    return video_id.values[0]

In [ ]:
def get_videos(data):
    video_array = []
    
    for d in data:
        d = find_video_id_by_track_id(d)
        video_array.append(d)
        
    return video_array

In [ ]:
y_true_videos = get_videos(true_y_track_values)